In [14]:
from preprocessing_class_v2 import *

train = pd.read_csv('C:/lg_capstone/find_abnormal/data/train_begin.csv')
test = pd.read_csv('C:/lg_capstone/find_abnormal/data/test_begin.csv')

In [15]:

def apply_final_preprocessing(class_instance):
    all_data = class_instance.data.copy()

    train_data = all_data[all_data['target'].notnull()]
    test_data = all_data[all_data['target'].isnull()]

    X_train, y_train = class_instance.final_preprocessing(train_data)
    X_test, _ = class_instance.final_preprocessing(test_data)

    # X_train, X_test에서 'Set ID' 컬럼 삭제
    X_train = X_train.drop('Set ID', axis=1)
    Set_ID_for_submission = X_test['Set ID']
    X_test = X_test.drop('Set ID', axis=1)

    # X_test의 컬럼 순서 맞추기
    X_test = X_test[X_train.columns]


    X_train, X_val, y_train, y_val = train_test_split(X_train , y_train, test_size=0.2, random_state=42)
    
    # SMOTE 적용
    smote = SMOTE(random_state=42)
    X_train, y_train = smote.fit_resample(X_train, y_train)


    return X_train, X_val, y_train, y_val, X_test, Set_ID_for_submission

In [16]:
preprocessing = Preprocessing(train, test)
preprocessing.dam()
preprocessing.fill1()
preprocessing.fill2()
preprocessing.autoclave()


submission_diff = preprocessing.submission_diff

c:\lg_capstone\find_abnormal\preprocessing_class_v2.py:130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  diff_test['target'] = 'AbNormal'


preprocessing - dam
preprocessing - fill1
preprocessing - fill2


c:\lg_capstone\find_abnormal\preprocessing_class_v2.py:396: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2'].replace(304.8, 305.0, inplace=True) # 304.8을 305.0으로 대체
c:\lg_capstone\find_abnormal\preprocessing_class_v2.py:397: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on wh

preprocessing - autoclave


In [17]:
X_train, X_val, y_train, y_val, X_test, Set_ID = apply_final_preprocessing(preprocessing)

Final preprocessing
라벨 인코딩 된 객체형 컬럼: Dispense Volume Collect Result_Fill1
라벨 인코딩 된 객체형 컬럼: HEAD NORMAL COORDINATE X AXIS(1, 2, 3)_Fill1
라벨 인코딩 된 객체형 컬럼: CURE TRACK POSITION X_Fill2
라벨 인코딩 된 객체형 컬럼: HEAD NORMAL COORDINATE X AXIS_Fill2
dropped에 추가된 컬럼:  Set ID
dropped에 추가된 컬럼:  HEAD NORMAL COORDINATE X AXIS(1, 2, 3)_Fill1
dropped에 추가된 컬럼:  Dispense Volume Collect Result_Fill1
X shape:  (40413, 72)
y shape:  (40413,)
Final preprocessing
라벨 인코딩 된 객체형 컬럼: Dispense Volume Collect Result_Fill1
라벨 인코딩 된 객체형 컬럼: HEAD NORMAL COORDINATE X AXIS(1, 2, 3)_Fill1
라벨 인코딩 된 객체형 컬럼: CURE TRACK POSITION X_Fill2
라벨 인코딩 된 객체형 컬럼: HEAD NORMAL COORDINATE X AXIS_Fill2
dropped에 추가된 컬럼:  Set ID
dropped에 추가된 컬럼:  HEAD NORMAL COORDINATE X AXIS(1, 2, 3)_Fill1
dropped에 추가된 컬럼:  Dispense Volume Collect Result_Fill1
X shape:  (17333, 72)
y shape:  (17333,)
Empty DataFrame
Columns: [Equipment_Dam, Model.Suffix_Dam, Workorder_Dam, DISCHARGED SPEED OF RESIN Collect Result_Dam, DISCHARGED TIME OF RESIN(Stage1) Collect Res

In [19]:
# 1. SMOTE를 적용하여 학습한 랜덤 포레스트 모델
model_weighted_smote = RandomForestClassifier(n_estimators=200, random_state=42)
# model_weighted_smote = RandomForestClassifier(n_estimators=200, random_state=42, class_weight={0: 1, 1: 10})

model_weighted_smote.fit(X_train, y_train)

# 2. 검증 데이터셋에서 예측 수행
y_pred_weighted_val = model_weighted_smote.predict(X_val)

# 3. 성능 평가
accuracy_weighted_val = accuracy_score(y_val, y_pred_weighted_val)
classification_rep_weighted_val = classification_report(y_val, y_pred_weighted_val, digits=4)

print("SMOTE 적용 모델 성능")
print("Accuracy:", accuracy_weighted_val)
print("Classification Report:\n", classification_rep_weighted_val)

# 4. 테스트 데이터에서 예측 수행
y_pred_test = model_weighted_smote.predict(X_test)

SMOTE 적용 모델 성능
Accuracy: 0.9465544970926636
Classification Report:
               precision    recall  f1-score   support

           0     0.6364    0.0476    0.0886       441
           1     0.9478    0.9984    0.9725      7642

    accuracy                         0.9466      8083
   macro avg     0.7921    0.5230    0.5305      8083
weighted avg     0.9308    0.9466    0.9242      8083



In [20]:
# 6. 결과를 DataFrame으로 저장
submission_same = pd.DataFrame({
    'Set ID': Set_ID.values,  # Set ID를 원래대로 유지
    'target': y_pred_test
})

# 기존 0과 1 값을 'AbNormal'과 'Normal'로 변환
submission_same['target'] = submission_same['target'].map({0: 'AbNormal', 1: 'Normal'})

# submission_same, submission_diff 데이터프레임을 하나로 합치기
submission = pd.concat([submission_same, submission_diff])

In [12]:
submission.to_csv('C:/lg_capstone/find_abnormal/data/submission.csv', index=False)

In [21]:
unique, counts = np.unique(submission_same['target'], return_counts=True)
print(unique, counts)

['AbNormal' 'Normal'] [   45 17288]
